## Importar bibliotecas
<br> Começaremos importando as bibliotecas Python que usaremos nesta análise. Essas bibliotecas incluem:
<ul>
<li> <b> pyodbc </b> drive odbc para carregar os dados. </li>
<li> <b> pandas </b> e <b> numpy </b> para ingestão e manipulação de dados. </li>
<li> <b> matplotlib </b> e <b> seaborn </b> para visualização de dados </li>
<li> <b> missingno </b>, que contém funções que exibem valores ausentes </li>

</ul>

In [1]:
#Bibliotecas para conexão a base de dados via script SQL
import pyodbc

#Bibliotecas para criação e manipulação de DATAFRAMES e Algebra 
import pandas as pd 
import numpy as np

#Bibliotecas para geração de gráficos
import matplotlib.pyplot as plt 
import seaborn as sns
import missingno as msno
import pylab

#### Ingerir dados

* Agora, precisaremos ler o conjunto de dados usando os comandos abaixo.

<b> Nota: </b> Certifique-se de executar a célula de importação acima (shift + enter) antes de executar o código de inserção de dados abaixo.


<h6>Conexão com base WIT Claro Chat 

In [2]:
odbc_mysql = '{MySQL ODBC 8.0 ANSI driver}'

# Conection MYSQL Server WIT
con_wit = pyodbc.connect(DRIVER=odbc_mysql, 
                         SERVER='localhost', 
                         PORT='6446', 
                         USER='adriano.fonseca', 
                         PASSWORD='Mudar01*', 
                         charset='utf8')

### Leads - Conduz o Cliente no Chat Claro

In [3]:
query = ('''
SELECT DISTINCT
    l.id_contact, l.phone, plan_type
FROM
    lead.leads AS l
WHERE
    l.is_claro = 1
        AND l.data LIKE '%Claro Chat%'
        AND l.id_campaign = 25
        AND l.id_contact IS NOT NULL
        AND l.created_at BETWEEN '2019-11-01 00:00:00' AND '2019-11-14 23:59:59';
''')
df_log = pd.read_sql_query(query, con_wit)

* A base de Leads está sendo carregada, pois devemos retorna todos os clientes pertinentes ao filtro para recuperação dos dados enriquecidos em recargas e serviços

### Caracteristicas | Análise

* id_contact | É o identificador do cliente na base, seremos norteados pela concatenação
* phone | Telefone do cliente na base, atraves dele, podemos tirar insights de localização do cliente
* plan_type | Tipo do plano do cliente, Clientes = PRE, Migra. Up clientes != de PRE

In [4]:
query = ('''
SELECT DISTINCT
    l.id_contact,
    l.phone,
    (IF(LEFT(l.phone, 2) LIKE '1%',
        'SP',
        IF(LEFT(l.phone, 2) IN ('21' , '22', '24', '27', '28'),
            'RJES',
            IF(LEFT(l.phone, 2) LIKE '3%',
                'MG',
                IF(LEFT(l.phone, 2) LIKE '4%',
                    'PRSC',
                    IF(LEFT(l.phone, 2) LIKE '5%',
                        'RS',
                        IF(LEFT(l.phone, 2) LIKE '6%',
                            'CO',
                            IF(LEFT(l.phone, 2) LIKE '7%',
                                'BASE',
                                IF(LEFT(l.phone, 2) LIKE '8%',
                                    'NE',
                                    'NO'))))))))) AS regional,
    (CASE
        WHEN c.birthday IS NULL THEN - 1
        WHEN
            YEAR(CURDATE()) - YEAR(c.birthday) < 0
                OR YEAR(CURDATE()) - YEAR(c.birthday) > 99
        THEN
            - 2
        ELSE YEAR(CURDATE()) - YEAR(c.birthday)
    END) AS idade_cliente,
    plan_type
FROM
    lead.leads AS l
        LEFT JOIN
    customer.customers AS c ON c.id = l.id_customer
WHERE
    l.is_claro = 1
        AND l.data LIKE '%Claro Chat%'
        AND l.id_campaign = 25
        AND l.id_contact IS NOT NULL
        AND l.created_at BETWEEN '2019-11-01 00:00:00' AND '2019-11-14 23:59:59';
''')
df_rf = pd.read_sql_query(query, con_wit)

* A base de Leads para Random Forest está sendo carregada, pois devemos retorna todos os clientes pertinentes ao filtro para recuperação dos dados enriquecidos em recargas e serviços

### Caracteristicas | Análise

* id_contact | É o identificador do cliente na base, seremos norteados pela concatenação
* regional | Feature da Região do cliente
* idade | Feature da Idade do cliente
* phone | Telefone do cliente na base, atraves dele, podemos tirar insights de localização do cliente
* plan_type | Tipo do plano do cliente, Clientes = PRE, Migra. Up clientes != de PRE

### Coluna Abandono - Informa quantas vezes o cliente abandonou o Chat

In [5]:
query = ('''
SELECT 
    customers.id_contact, customers.id_customer
FROM
    abandonment.customers AS customers
WHERE
    customers.id_contact IS NOT NULL
        AND customers.id_campaign = 25
        AND customers.params LIKE '%Claro Chat%'
        AND customers.created_at BETWEEN '2019-11-01 00:00:00' AND '2019-11-14 23:59:59'
ORDER BY customers.id_contact DESC;
''')
df1 = pd.read_sql_query(query, con_wit)

In [6]:
ab = df1.groupby(['id_contact']).count().reset_index()

In [7]:
df1 = ab[ab['id_customer']>=2]

In [8]:
df1 = df1[['id_customer','id_contact']]

In [9]:
df1.rename(columns={"id_customer":"Qnt_abandono"}, inplace=True)

### Caracteristicas | Análise
Qnt_abandono | Contador de quantas vezes o cliente abandonou o Chat

### Features de Recargas - Recuperar as features de recarga enriquecidas.

In [10]:
query = ('''
SELECT DISTINCT
    rec.id_contact,
    SUM(rec.value) AS sum_recharge,
    COUNT(1) AS recharge_frequency
FROM
    (SELECT DISTINCT
        id_contact, `type`, value, date_time
    FROM
        carrier.costs
    WHERE
        id_contact IN (SELECT 
                id_contact
            FROM
                lead.leads
            WHERE
                is_claro = 1
                    AND data LIKE '%Claro Chat%'
                    AND id_campaign = 25
                    AND id_contact IS NOT NULL
                    AND created_at BETWEEN '2019-11-01 00:00:00' AND '2019-11-14 23:59:59')) AS rec
GROUP BY rec.id_contact;
''')
df2 = pd.read_sql_query(query, con_wit)

### Caracteristicas | Análise

* sum_recharge | Coluna com a soma da recarga do cliente
* recharge_frequency | Coluna que traz a frequencia de recarga do cliente

In [11]:
query = ('''
SELECT 
    *
FROM
    (SELECT DISTINCT
        rec.id_contact,
            rec_online_10,
            rec_online_35_b5,
            rec_online_15,
            sos_rec_5,
            rec_online_20_b2,
            chip_pre_rec_10,
            chip_pre_rec_20,
            rec_online_13,
            rec_online_50_b8,
            rec_online_30_b4,
            rec_online_40_b6,
            pct_rec_1190,
            pct_rec_690,
            rec_online_100_b18,
            pct_rec_sos_5,
            sos_rec_3,
            rec_online_8
    FROM
        (SELECT DISTINCT
        rec.id_contact,
            SUM(rec.rec_online_10) AS rec_online_10,
            SUM(rec.rec_online_35_b5) AS rec_online_35_b5,
            SUM(rec.rec_online_15) AS rec_online_15,
            SUM(rec.sos_rec_5) AS sos_rec_5,
            SUM(rec.rec_online_20_b2) AS rec_online_20_b2,
            SUM(rec.chip_pre_rec_10) AS chip_pre_rec_10,
            SUM(rec.chip_pre_rec_20) AS chip_pre_rec_20,
            SUM(rec.rec_online_13) AS rec_online_13,
            SUM(rec.rec_online_50_b8) AS rec_online_50_b8,
            SUM(rec.rec_online_30_b4) AS rec_online_30_b4,
            SUM(rec.rec_online_40_b6) AS rec_online_40_b6,
            SUM(rec.pct_rec_1190) AS pct_rec_1190,
            SUM(rec.pct_rec_690) AS pct_rec_690,
            SUM(rec.rec_online_100_b18) AS rec_online_100_b18,
            SUM(rec.pct_rec_sos_5) AS pct_rec_sos_5,
            SUM(rec.sos_rec_3) AS sos_rec_3,
            SUM(rec.rec_online_8) AS rec_online_8
    FROM
        (SELECT DISTINCT
        rec.id_contact,
            rec.date_time,
            IF(rec.type LIKE '%RECARGA ONLINE R$10%', 1, 0) AS rec_online_10,
            IF(rec.type LIKE '%RECARGA ONLINE R$35 + BONUS R$5%', 1, 0) AS rec_online_35_b5,
            IF(rec.type LIKE '%RECARGA ONLINE R$15%', 1, 0) AS rec_online_15,
            IF(rec.type LIKE '%SOS RECARGA - R$5%', 1, 0) AS sos_rec_5,
            IF(rec.type LIKE '%RECARGA ONLINE R$20 + BONUS R$2%', 1, 0) AS rec_online_20_b2,
            IF(rec.type LIKE '%CHIP PRE + RECARGA R$10%', 1, 0) AS chip_pre_rec_10,
            IF(rec.type LIKE '%CHIP PRE + RECARGA R$20%', 1, 0) AS chip_pre_rec_20,
            IF(rec.type LIKE '%RECARGA ONLINE R$13%', 1, 0) AS rec_online_13,
            IF(rec.type LIKE '%RECARGA ONLINE R$50 + BONUS R$8%', 1, 0) AS rec_online_50_b8,
            IF(rec.type LIKE '%RECARGA ONLINE R$30 + BONUS R$4%', 1, 0) AS rec_online_30_b4,
            IF(rec.type LIKE '%RECARGA ONLINE R$40 + BONUS R$6%', 1, 0) AS rec_online_40_b6,
            IF(rec.type LIKE '%PACOTE RECARGA R$11,90%', 1, 0) AS pct_rec_1190,
            IF(rec.type LIKE '%PACOTE RECARGA R$6,90%', 1, 0) AS pct_rec_690,
            IF(rec.type LIKE '%RECARGA ONLINE R$100 + BONUS R$18%', 1, 0) AS rec_online_100_b18,
            IF(rec.type LIKE '%PACOTE SOS RECARGA R$5,00%', 1, 0) AS pct_rec_sos_5,
            IF(rec.type LIKE '%SOS RECARGA - R$3%', 1, 0) AS sos_rec_3,
            IF(rec.type LIKE '%RECARGA ONLINE R$8%', 1, 0) AS rec_online_8
    FROM
        (SELECT DISTINCT
        id_contact, `type`, value, date_time
    FROM
        carrier.costs
    WHERE
        id_contact IN (SELECT 
                id_contact
            FROM
                lead.leads
            WHERE
                is_claro = 1
                    AND data LIKE '%Claro Chat%'
                    AND id_campaign = 25
                    AND id_contact IS NOT NULL
                    AND created_at BETWEEN '2019-11-01 00:00:00' AND '2019-11-14 23:59:59')) AS rec
    GROUP BY rec.id_contact , rec.type , rec.date_time) AS rec
    GROUP BY rec.id_contact) AS rec) AS tab;
''')
df3 = pd.read_sql_query(query, con_wit)

### Caracteristicas | Análise

* rec_online_10 | Feature que representa o contador de quantas vezes o cliente utilizou a recarga online de 10 reais
* rec_online_35_b5 | Feature que representa o contador de quantas vezes o cliente utilizou a recarga online de 35 reais + bonus 
* rec_online_15 | Feature que representa o contador de quantas vezes o cliente utilizou a recarga online de 15 reais
* sos_rec_5 | Feature que representa o contador de quantas vezes o cliente utilizou a recarga sos 5 reais
* rec_online_20_b2 | Feature que representa o contador de quantas vezes o cliente utilizou a recarga online de 20 reais + bonus
* chip_pre_rec_10 | Feature que representa o contador de quantas vezes o cliente utilizou a recarga chip pre 10 reais
* crip_pre_rec_20 | Feature que representa o contador de quantas vezes o cliente utilizou a recarga chip pre 20 reais
* rec_online_13 | Feature que representa o contador de quantas vezes o cliente utilizou a recarga online 13 reais
* rec_online_50_b8 | Feature que representa o contador de quantas vezes o cliente utilizou a recarga online 50 reais + bonus
* rec_online_30_b4 | Feature que representa o contador de quantas vezes o cliente utilizou a recarga online 30 reais + bonus
* rec_online_40_b6 | Feature que representa o contador de quantas vezes o cliente utilizou a recarga online 40 reais + bonus
* pct_rec_1190 | Feature que representa o contador de quantas vezes o cliente utilizou a recarga 12 reais
* pct_rec_690 | Feature que representa o contador de quantas vezes o cliente utilizou a recarga de 7 reais
* rec_online_100_b18 | Feature que representa o contador de quantas vezes o cliente utilizou recarga online 100 reais + bonus
* pct_rec_sos_5 | Feature que representa o contador de quantas vezes o cliente utilizou pacote recarga sos 5 reais
* sos_rec_3 | Feature que representa o contador de quantas vezes o cliente utilizou recarga sos 3 reais
* rec_online_8 | Feature que representa o contador de quantas vezes o cliente utilizou recarga online 8 reais

### Serviços - Recuperar as features de recarga enriquecidas.

In [12]:
query = ('''
SELECT DISTINCT
    serv.id_contact,
    SUM(serv.value) AS sum_services,
    COUNT(1) AS services_frequency
FROM
    (SELECT DISTINCT
        id_contact, description, value, date_time
    FROM
        carrier.services
    WHERE
        id_contact IN (SELECT 
                id_contact
            FROM
                lead.leads
            WHERE
                is_claro = 1
                    AND data LIKE '%Claro Chat%'
                    AND id_campaign = 25
                    AND id_contact IS NOT NULL
                    AND created_at BETWEEN '2019-11-01 00:00:00' AND '2019-11-14 23:59:59')) AS serv
GROUP BY serv.id_contact;
''')
df4 = pd.read_sql_query(query, con_wit)

In [13]:
query = ('''
SELECT 
    *
FROM
    (SELECT DISTINCT
        serv.id_contact,
            inter_avulsa,
            antivirus,
            app_educacao,
            app_emprego,
            app_saude,
            claro_clube,
            claro_pre_mix_giga,
            entretenimento,
            games,
            pct_internet_mensal,
            prezao_diario,
            prezao_mensal,
            prezao_quinzenal,
            prezao_semanal,
            recarga_sos,
            servicos_operadora,
            set1,
            sms_cobrar,
            sms_internacional,
            transf_entre_regionais,
            truecaller
    FROM
        (SELECT DISTINCT
        serv.id_contact,
            SUM(serv.inter_avulsa) AS inter_avulsa,
            SUM(serv.antivirus) AS antivirus,
            SUM(serv.app_educacao) AS app_educacao,
            SUM(serv.app_emprego) AS app_emprego,
            SUM(serv.app_saude) AS app_saude,
            SUM(serv.claro_clube) AS claro_clube,
            SUM(serv.claro_pre_mix_giga) AS claro_pre_mix_giga,
            SUM(serv.entretenimento) AS entretenimento,
            SUM(serv.games) AS games,
            SUM(serv.pct_internet_mensal) AS pct_internet_mensal,
            SUM(serv.prezao_diario) AS prezao_diario,
            SUM(serv.prezao_mensal) AS prezao_mensal,
            SUM(serv.prezao_quinzenal) AS prezao_quinzenal,
            SUM(serv.prezao_semanal) AS prezao_semanal,
            SUM(serv.recarga_sos) AS recarga_sos,
            SUM(serv.servicos_operadora) AS servicos_operadora,
            SUM(serv.set1) AS set1,
            SUM(serv.sms_cobrar) AS sms_cobrar,
            SUM(serv.sms_internacional) AS sms_internacional,
            SUM(serv.transf_entre_regionais) AS transf_entre_regionais,
            SUM(serv.truecaller) AS truecaller
    FROM
        (SELECT DISTINCT
        serv.id_contact,
            serv.date_time,
            IF(serv.description LIKE '%internet avulsa%'
                OR serv.description LIKE '%PACOTE DIARIO 100MB + WHATSAPP AD%'
                OR serv.description LIKE '%PACOTE DIARIO 200MB + WHATSAPP AD%'
                OR serv.description LIKE '%PCT 500MB MAIS 7 DIAS%'
                OR serv.description LIKE '%PCT ADICIONAL 1GB 7 DIAS%'
                OR serv.description LIKE '%PCT DIARIO 100MB + WHATSAPP%'
                OR serv.description LIKE '%PCT DIARIO 200MB + WHATSAPP%'
                OR serv.description LIKE '%TUDO POR R$1.49/DIA%', 1, 0) AS inter_avulsa,
            IF(serv.description LIKE '%NORTON%'
                OR serv.description LIKE '%SEGURANCA HERO%', 1, 0) AS antivirus,
            IF(serv.description LIKE '%CLARO IDIOMAS BY BUSUU%'
                OR serv.description LIKE '%FUN ENGLISH WITH DOKI%'
                OR serv.description LIKE '%INGLES MAGICO APP%', 1, 0) AS app_educacao,
            IF(serv.description LIKE '%MINUTO CARREIRA%', 1, 0) AS app_emprego,
            IF(serv.description LIKE '%BTFIT%', 1, 0) AS app_saude,
            IF(serv.description LIKE '%CLARO CONECTA%'
                OR serv.description LIKE '%CLARO FOTO%'
                OR serv.description LIKE '%CLARO FOTO - OUTRAS OPERADORAS%'
                OR serv.description LIKE '%CLARO FOTO PARA E-MAIL%'
                OR serv.description LIKE '%CLARO GAMES%'
                OR serv.description LIKE '%CLARO NOTICIAS%'
                OR serv.description LIKE '%CLUBE CLARO APPS%', 1, 0) AS claro_clube,
            IF(serv.description LIKE '%CLARO PRE MIX GIGA%', 1, 0) AS claro_pre_mix_giga,
            IF(serv.description LIKE '%BADOO - COMPRA AVULSA%'
                OR serv.description LIKE '%BAND SPORTS%'
                OR serv.description LIKE '%CLUBE CLARO KIDS%'
                OR serv.description LIKE '%CLUBE MARVEL%'
                OR serv.description LIKE '%CLUBE STARWARS%'
                OR serv.description LIKE '%CUPIDOO%'
                OR serv.description LIKE '%DESCOMPLICA%'
                OR serv.description LIKE '%DISCOVERY KIDS ON%'
                OR serv.description LIKE '%DISNEY CUBES%'
                OR serv.description LIKE '%EGO%'
                OR serv.description LIKE '%FOX SPORTS GOL%'
                OR serv.description LIKE '%JOGADA MUSICAL%'
                OR serv.description LIKE '%JOGADA DE PREMIOS%'
                OR serv.description LIKE '%LIVROH%'
                OR serv.description LIKE '%MIX DE  VANTAGENS%'
                OR serv.description LIKE '%PASSATEMPO PREMIADO%'
                OR serv.description LIKE '%PET DICAS%'
                OR serv.description LIKE '%PLAYKIDS%'
                OR serv.description LIKE '%PLAY KIDS%'
                OR serv.description LIKE '%POCOYO HOUSE%'
                OR serv.description LIKE '%PRIME TUBE%'
                OR serv.description LIKE '%QUERO DESCONTOS%'
                OR serv.description LIKE '%RADAR%'
                OR serv.description LIKE '%SOCIAL COMICS%'
                OR serv.description LIKE '%SONHO PREMIADO%'
                OR serv.description LIKE '%SX REVOLUTION%'
                OR serv.description LIKE '%UBOOK%'
                OR serv.description LIKE '%UMBARATO%'
                OR serv.description LIKE '%UOL CURSO DE BOLSO%'
                OR serv.description LIKE '%VIAJE MAIS%'
                OR serv.description LIKE '%VOCE GOURMET%', 1, 0) AS entretenimento,
            IF(serv.description LIKE '%APP GAME%'
                OR serv.description LIKE '%FUTCEL%'
                OR serv.description LIKE '%GAMELOFT ITEM DE JOGO%'
                OR serv.description LIKE '%GAMES FUN%'
                OR serv.description LIKE '%JOGOS DE SEMPRE%'
                OR serv.description LIKE '%NEYMAR JR. EXPERIENCE%'
                OR serv.description LIKE '%PROMO GENIUS GAME%'
                OR serv.description LIKE '%QUIZ SUPER - JOGO%', 1, 0) AS games,
            IF(serv.description LIKE '%PACOTE MENSAL 1GB + WHATSAPP GRATIS%'
                OR serv.description LIKE '%PACOTE MENSAL 2GB + WHATSAPP GRATIS%'
                OR serv.description LIKE '%PACOTE MENSAL 300MB + WHATSAPP GRATIS%'
                OR serv.description LIKE '%PACOTE MENSAL 500MB + WHATSAPP GRATIS%', 1, 0) AS pct_internet_mensal,
            IF(serv.description LIKE '%PREZAO DIARIO 100MB + WHATSAPP + SMS%'
                OR serv.description LIKE '%PREZAO DIARIO 100MB +WHATS+ SMS 1,99/DIA%', 1, 0) AS prezao_diario,
            IF(serv.description LIKE '%PREZAO 34,99 POR MES%'
                OR serv.description LIKE '%PREZAO 14,99 POR MES%'
                OR serv.description LIKE '%PREZAO R$19,99 POR MES%'
                OR serv.description LIKE '%PREZAO FALA MAIS MENSAL%', 1, 0) AS prezao_mensal,
            IF(serv.description LIKE '%PREZAO FALA MAIS 14 DIAS%'
                OR serv.description LIKE '%PREZAO MUITO MAIS 14 DIAS%', 1, 0) AS prezao_quinzenal,
            IF(serv.description LIKE '%PREZAO 9,99 POR SEMANA%'
                OR serv.description LIKE '%PREZAO 14,99 POR SEMANA%', 1, 0) AS prezao_semanal,
            IF(serv.description LIKE '%SOS RECARGA - DEBITO EMPRESTIMO APP%'
                OR serv.description LIKE '%SOS RECARGA - DEBITO R$2,00%'
                OR serv.description LIKE '%SOS RECARGA - DEBITO R$5,00%'
                OR serv.description LIKE '%SOS RECARGA - DEBITO TAXA APP%', 1, 0) AS recarga_sos,
            IF(serv.description LIKE '%ADESAO CHIP NATURA%'
                OR serv.description LIKE '%AJUSTES DE CREDITOS%'
                OR serv.description LIKE '%CLARO RECADO%'
                OR serv.description LIKE '%CLARO RECADO PREMIUM%'
                OR serv.description LIKE '%ESTORNO AUTOMATICO DE ANATEL 226%'
                OR serv.description LIKE '%ESTORNO TORPEDO%'
                OR serv.description LIKE '%EXPIRACAO DE CREDITO APOS 30 DIAS.%'
                OR serv.description LIKE '%RECADO AVULSO%'
                OR serv.description LIKE '%SOM DE CHAMADA%', 1, 0) AS servicos_operadora,
            IF(serv.description LIKE '%set%', 1, 0) AS set1,
            IF(serv.description LIKE '%TORPEDO A COBRAR%'
                OR serv.description LIKE '%TORPEDO A COBRAR DEGRAU OUTRAS OPER%', 1, 0) AS sms_cobrar,
            IF(serv.description LIKE '%TORPEDO - INTERNACIONAL%', 1, 0) AS sms_internacional,
            IF(serv.description LIKE '%TRANSFERENCIA ENTRE REGIONAIS%', 1, 0) AS transf_entre_regionais,
            IF(serv.description LIKE '%TRUECALLER%', 1, 0) AS truecaller
    FROM
        (SELECT DISTINCT
        id_contact, description, value, date_time
    FROM
        carrier.services
    WHERE
        id_contact IN (SELECT 
                id_contact
            FROM
                lead.leads
            WHERE
                is_claro = 1
                    AND data LIKE '%Claro Chat%'
                    AND id_campaign = 25
                    AND id_contact IS NOT NULL
                    AND created_at BETWEEN '2019-11-01 00:00:00' AND '2019-11-14 23:59:59')) AS serv
    GROUP BY serv.id_contact , serv.description , serv.date_time) AS serv
    GROUP BY serv.id_contact) AS serv) AS tab;
''')
df5 = pd.read_sql_query(query, con_wit)

### Caracteristicas | Análise

#### Tipos de serviços Agupados

* Serviços Operadora | Feature que representa o contador de quantas vezes o cliente utilizou serviços da operadora

* Games | Feature que representa o contador de quantas vezes o cliente utilizou serviços de games

* Entretenimento | Feature que representa o contador de quantas vezes o cliente utilizou serviços de entretenimento

* APP saúde | Feature que representa o contador de quantas vezes o cliente utilizou serviços app saude

* APP Educação | Feature que representa o contador de quantas vezes o cliente utilizou serviços app educação

* APP Emprego | Feature que representa o contador de quantas vezes o cliente utilizou serviços app emprego

* Claro Clube | Feature que representa o contador de quantas vezes o cliente utilizou serviços  serviços do claro clube

* CLARO PRE MIX GIGA | Feature que representa o contador de quantas vezes o cliente utilizou serviços claro pre mix giga

* Internet Avulsa | Feature que representa o contador de quantas vezes o cliente utilizou serviços internet avulsa

* Antivírus | Feature que representa o contador de quantas vezes o cliente utilizou serviços antivirus

* Pct. Minutos | Feature que representa o contador de quantas vezes o cliente utilizou serviços pacote de minutos

* Pct. Internet Mensal | Feature que representa o contador de quantas vezes o cliente utilizou serviços pacote internet mensal

* Pct. SMS | Feature que representa o contador de quantas vezes o cliente utilizou serviços pacote sms

* Prezão Mensal | Feature que representa o contador de quantas vezes o cliente utilizou serviços prezão mensal

* Prezão Semanal | Feature que representa o contador de quantas vezes o cliente utilizou serviços prezão semanal

* Prezão Diário | Feature que representa o contador de quantas vezes o cliente utilizou serviços prezão diario

* Prezão Quinzenal | Feature que representa o contador de quantas vezes o cliente utilizou serviços prezão quinzenal

* SET | Feature que representa o contador de quantas vezes o cliente utilizou serviços set

* SMS Internacional | Feature que representa o contador de quantas vezes o cliente utilizou serviços sms internacional

* Recarga SOS | Feature que representa o contador de quantas vezes o cliente utilizou serviços recarga sos

* SMS a cobrar | Feature que representa o contador de quantas vezes o cliente utilizou serviços sms a cobrar

* Transferência entre Regionais | Feature que representa o contador de quantas vezes o cliente utilizou serviços trans x Reg 

* TRUECALLER | Feature que representa o contador de quantas vezes o cliente utilizou serviços truecaller

### Vendas - Recuperar classes de venda.

In [14]:
query = ('''        
SELECT DISTINCT
    l.id_contact, IF(sl.id IS NULL, 0, 1) AS venda
FROM
    lead.leads AS l
        LEFT JOIN
    sale.sales AS sl ON l.id_contact = sl.id_contact
WHERE
    l.is_claro = 1
        AND l.data LIKE '%Claro Chat%'
        AND l.id_campaign = 25
        AND l.created_at BETWEEN '2019-11-01 00:00:00' AND '2019-11-14 23:59:59'
GROUP BY l.id_contact;
''')
df6 = pd.read_sql_query(query, con_wit)

### Classes de Venda

* venda | Coluna que obtem as classes de venda e não venda do dataset, pode ser considerada como objetivo do estudo.

### Merge dos datasets Logistic

In [15]:
data = pd.merge(df_log, df1, how = 'left', on=['id_contact'])
data = pd.merge(data, df2, how = 'left', on=['id_contact'])
data = pd.merge(data, df3, how = 'left', on=['id_contact'])
data = pd.merge(data, df4, how = 'left', on=['id_contact'])
data = pd.merge(data, df5, how = 'left', on=['id_contact'])
data = pd.merge(data, df6, how = 'left', on=['id_contact'])
data['Qnt_abandono'] = data['Qnt_abandono'].fillna(value=1)

In [16]:
data.to_csv('data_test_log.csv', sep=';', index=False)

### Merge dos datasets Random Forest

In [17]:
data = pd.merge(df_rf, df1, how = 'left', on=['id_contact'])
data = pd.merge(data, df2, how = 'left', on=['id_contact'])
data = pd.merge(data, df3, how = 'left', on=['id_contact'])
data = pd.merge(data, df4, how = 'left', on=['id_contact'])
data = pd.merge(data, df5, how = 'left', on=['id_contact'])
data = pd.merge(data, df6, how = 'left', on=['id_contact'])
data['Qnt_abandono'] = data['Qnt_abandono'].fillna(value=1)

In [18]:
data.to_csv('data_test_rf.csv', sep=';', index=False)